# Agent for beginners TASK 2 代码打卡

教程提供的代码稍微把LLM calling的部分改了下，用了自己习惯的模式。为了省钱用了gpt-5-nano。

代码没问题可以跑通。

最后让GPT优化了一版更结构化输出的代码，我自己好理解点。

In [ ]:
# code from hello-agents repo

!pip install requests tavily-python openai

In [2]:
AGENT_SYSTEM_PROMPT = """
你是一个智能旅行助手。你的任务是分析用户的请求，并使用可用工具一步步地解决问题。

# 可用工具:
- `get_weather(city: str)`: 查询指定城市的实时天气。
- `get_attraction(city: str, weather: str)`: 根据城市和天气搜索推荐的旅游景点。

# 行动格式:
你的回答必须严格遵循以下格式。首先是你的思考过程，然后是你要执行的具体行动，每次回复只输出一对Thought-Action：
Thought: [这里是你的思考过程和下一步计划]
Action: [这里是你要调用的工具，格式为 function_name(arg_name="arg_value")]

# 任务完成:
当你收集到足够的信息，能够回答用户的最终问题时，你必须在`Action:`字段后使用 `finish(answer="...")` 来输出最终答案。

请开始吧！
"""


In [3]:
import requests
import json

def get_weather(city: str) -> str:
    """
    通过调用 wttr.in API 查询真实的天气信息。
    """
    # API端点，我们请求JSON格式的数据
    url = f"https://wttr.in/{city}?format=j1"
    
    try:
        # 发起网络请求
        response = requests.get(url)
        # 检查响应状态码是否为200 (成功)
        response.raise_for_status() 
        # 解析返回的JSON数据
        data = response.json()
        
        # 提取当前天气状况
        current_condition = data['current_condition'][0]
        weather_desc = current_condition['weatherDesc'][0]['value']
        temp_c = current_condition['temp_C']
        
        # 格式化成自然语言返回
        return f"{city}当前天气:{weather_desc}，气温{temp_c}摄氏度"
        
    except requests.exceptions.RequestException as e:
        # 处理网络错误
        return f"错误:查询天气时遇到网络问题 - {e}"
    except (KeyError, IndexError) as e:
        # 处理数据解析错误
        return f"错误:解析天气数据失败，可能是城市名称无效 - {e}"


In [4]:
import os
from tavily import TavilyClient

def get_attraction(city: str, weather: str) -> str:
    """
    根据城市和天气，使用Tavily Search API搜索并返回优化后的景点推荐。
    """
    # 1. 从环境变量中读取API密钥
    api_key = os.environ.get("TAVILY_API_KEY")
    if not api_key:
        return "错误:未配置TAVILY_API_KEY环境变量。"

    # 2. 初始化Tavily客户端
    tavily = TavilyClient(api_key=api_key)
    
    # 3. 构造一个精确的查询
    query = f"'{city}' 在'{weather}'天气下最值得去的旅游景点推荐及理由"
    
    try:
        # 4. 调用API，include_answer=True会返回一个综合性的回答
        response = tavily.search(query=query, search_depth="basic", include_answer=True)
        
        # 5. Tavily返回的结果已经非常干净，可以直接使用
        # response['answer'] 是一个基于所有搜索结果的总结性回答
        if response.get("answer"):
            return response["answer"]
        
        # 如果没有综合性回答，则格式化原始结果
        formatted_results = []
        for result in response.get("results", []):
            formatted_results.append(f"- {result['title']}: {result['content']}")
        
        if not formatted_results:
             return "抱歉，没有找到相关的旅游景点推荐。"

        return "根据搜索，为您找到以下信息:\n" + "\n".join(formatted_results)

    except Exception as e:
        return f"错误:执行Tavily搜索时出现问题 - {e}"


In [5]:
# 将所有工具函数放入一个字典，方便后续调用
available_tools = {
    "get_weather": get_weather,
    "get_attraction": get_attraction,
}


In [ ]:
from __future__ import annotations

import os
import json
import traceback
from dataclasses import dataclass
from typing import Any, Callable, Dict, Optional, Tuple

from openai import OpenAI


AGENT_SYSTEM_PROMPT = r"""
你是一个严格遵守协议的 Agent。
你必须且只能输出一个 JSON 对象，不允许输出任何额外文本，不允许使用 Markdown 代码块。

JSON 格式必须是：
{
  "thought": "简短思考（字段必须存在）",
  "action": {
    "tool": "工具名，必须来自工具列表",
    "args": { "参数名": 参数值, ... }
  }
}

规则：
1 每次只允许输出一次 action，不允许输出多个 action
2 tool 必须从工具列表中选择
3 信息不足时优先调用 get_weather 或 search_web
4 可以给最终答复时必须调用 finish 且 args 必须包含 answer

工具列表：
get_weather
search_web
recommend_spot
finish
""".strip()


@dataclass
class AgentConfig:
    max_loops: int = 6
    debug_print: bool = True
    json_repair_retries: int = 2


class OpenAIAdapter:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model

    def generate(self, prompt: str, system_prompt: str) -> str:
        resp = self.client.responses.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
        )
        return resp.output_text


def tool_get_weather(city: str, date: str = "today") -> Dict[str, Any]:
    # 示例返回，接真实天气 API 时在这里替换
    return {
        "city": city,
        "date": date,
        "condition": "晴",
        "temperature_c": 2,
        "wind": "西北风 3 级",
        "humidity_percent": 35,
    }


def tool_search_web(query: str) -> Dict[str, Any]:
    # 示例返回，接 Tavily 或其它搜索时在这里替换
    return {
        "query": query,
        "results": [
            {"title": "示例结果 1", "snippet": "这里是摘要", "url": "https://example.com/1"},
            {"title": "示例结果 2", "snippet": "这里是摘要", "url": "https://example.com/2"},
        ],
    }


def tool_recommend_spot(city: str, weather: Dict[str, Any]) -> Dict[str, Any]:
    condition = str(weather.get("condition", ""))
    temp = weather.get("temperature_c", None)

    if "雨" in condition:
        spot = "国家博物馆或大型室内展馆"
        reason = "下雨更适合室内，体验稳定"
    else:
        if isinstance(temp, (int, float)) and temp <= 3:
            spot = "什刹海周边短时散步，搭配室内茶馆休息"
            reason = "晴但偏冷，短户外加室内更舒服"
        else:
            spot = "颐和园或奥林匹克森林公园"
            reason = "晴朗适合大面积户外景点"

    return {
        "city": city,
        "spot": spot,
        "reason": reason,
        "based_on": {"condition": condition, "temperature_c": temp},
    }


def tool_finish(answer: str) -> Dict[str, Any]:
    return {"answer": answer}


TOOLS: Dict[str, Callable[..., Any]] = {
    "get_weather": tool_get_weather,
    "search_web": tool_search_web,
    "recommend_spot": tool_recommend_spot,
    "finish": tool_finish,
}


def parse_agent_json(output: str) -> Tuple[Optional[Dict[str, Any]], Optional[str]]:
    try:
        data = json.loads(output)
    except json.JSONDecodeError as e:
        return None, f"JSONDecodeError: {e}"

    if not isinstance(data, dict):
        return None, "协议错误 顶层必须是 JSON object"
    if "thought" not in data or "action" not in data:
        return None, "协议错误 必须包含 thought 和 action"
    if not isinstance(data["action"], dict):
        return None, "协议错误 action 必须是 object"
    if "tool" not in data["action"] or "args" not in data["action"]:
        return None, "协议错误 action 必须包含 tool 和 args"
    if not isinstance(data["action"]["tool"], str) or not data["action"]["tool"]:
        return None, "协议错误 tool 必须是非空字符串"
    if not isinstance(data["action"]["args"], dict):
        return None, "协议错误 args 必须是 object"

    return data, None


class JSONReActAgent:
    def __init__(self, llm: Any, cfg: AgentConfig, tools: Dict[str, Callable[..., Any]], system_prompt: str):
        self.llm = llm
        self.cfg = cfg
        self.tools = tools
        self.system_prompt = system_prompt

    def _log(self, s: str) -> None:
        if self.cfg.debug_print:
            print(s)

    def _render_history(self, history: list[Dict[str, str]]) -> str:
        lines: list[str] = []
        for m in history:
            role = m["role"]
            content = m["content"]
            if role == "user":
                lines.append(f"User: {content}")
            elif role == "assistant":
                lines.append(f"Assistant: {content}")
            else:
                lines.append(f"Observation: {content}")
        return "\n".join(lines)

    def run(self, user_prompt: str) -> str:
        history: list[Dict[str, str]] = [{"role": "user", "content": user_prompt}]

        for i in range(self.cfg.max_loops):
            self._log(f"\n======= LOOP {i + 1} =======")
            full_prompt = self._render_history(history)

            raw = self.llm.generate(full_prompt, system_prompt=self.system_prompt)
            self._log(f"LLM raw:\n{raw}")

            data, err = parse_agent_json(raw)

            repair = 0
            while err is not None and repair < self.cfg.json_repair_retries:
                repair += 1
                fix_obs = {
                    "error": "你的输出不符合协议 只能输出一个合法 JSON object 且包含 thought action tool args",
                    "detail": err,
                    "example": {
                        "thought": "need weather",
                        "action": {"tool": "get_weather", "args": {"city": "北京", "date": "today"}},
                    },
                }
                history.append({"role": "assistant", "content": raw})
                history.append({"role": "tool", "content": json.dumps(fix_obs, ensure_ascii=False)})

                full_prompt = self._render_history(history)
                raw = self.llm.generate(full_prompt, system_prompt=self.system_prompt)
                self._log(f"LLM repaired:\n{raw}")
                data, err = parse_agent_json(raw)

            if err is not None:
                return f"失败 模型多次输出无效 JSON 最后错误 {err}"

            action = data["action"]
            tool = action["tool"]
            args = action["args"]

            if tool not in self.tools:
                obs = {"error": f"未定义工具 {tool}", "allowed_tools": list(self.tools.keys())}
                history.append({"role": "assistant", "content": raw})
                history.append({"role": "tool", "content": json.dumps(obs, ensure_ascii=False)})
                continue

            try:
                result = self.tools[tool](**args)
            except TypeError as e:
                obs = {
                    "error": "工具调用参数错误",
                    "detail": str(e),
                    "tool": tool,
                    "args": args,
                }
                history.append({"role": "assistant", "content": raw})
                history.append({"role": "tool", "content": json.dumps(obs, ensure_ascii=False)})
                continue
            except Exception as e:
                obs = {
                    "error": "工具执行异常",
                    "detail": str(e),
                    "traceback": traceback.format_exc()[:2000],
                    "tool": tool,
                }
                history.append({"role": "assistant", "content": raw})
                history.append({"role": "tool", "content": json.dumps(obs, ensure_ascii=False)})
                continue

            self._log(f"Tool result:\n{json.dumps(result, ensure_ascii=False, indent=2)}")

            if tool == "finish":
                ans = result.get("answer", "")
                return ans if ans else "finish 未提供 answer"

            history.append({"role": "assistant", "content": raw})
            history.append({"role": "tool", "content": json.dumps(result, ensure_ascii=False)})

        return "已达到最大循环次数 仍未 finish"


def main() -> None:
    api_key = os.getenv("OPENAI_API_KEY")
    base_url = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
    model = os.getenv("OPENAI_MODEL", "gpt-5-nano")

    if not api_key:
        raise RuntimeError("OPENAI_API_KEY 未设置")

    client = OpenAI(api_key=api_key, base_url=base_url)
    llm = OpenAIAdapter(client=client, model=model)

    agent = JSONReActAgent(
        llm=llm,
        cfg=AgentConfig(max_loops=6, debug_print=True, json_repair_retries=2),
        tools=TOOLS,
        system_prompt=AGENT_SYSTEM_PROMPT,
    )

    user_prompt = "你好，请帮我查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点。"
    print("用户输入:", user_prompt)

    answer = agent.run(user_prompt)
    print("\nFINAL ANSWER:")
    print(answer)


if __name__ == "__main__":
    main()


ImportError: cannot import name 'OpenAICompatibleClient' from 'openai' (d:\Python313\Lib\site-packages\openai\__init__.py)

In [9]:
API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
MODEL_ID = os.getenv("MODEL_ID", "gpt-5-mini")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

print("API_KEY:", API_KEY[:10] + "..." if API_KEY else "Not found")
print("BASE_URL:", BASE_URL)
print("MODEL_ID:", MODEL_ID)
print("TAVILY_API_KEY:", TAVILY_API_KEY[:10] + "..." if TAVILY_API_KEY else "Not found")

API_KEY: sk-d7GDhmo...
BASE_URL: https://xiaoai.plus/v1
MODEL_ID: gpt-5-mini
TAVILY_API_KEY: tvly-dev-l...


In [17]:
import re
# --- 2. 初始化 ---

user_prompt = "你好，请帮我查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点。"
prompt_history = [f"用户请求: {user_prompt}"]

print(f"用户输入: {user_prompt}\n" + "="*40)

# --- 3. 运行主循环 ---
for i in range(5): # 设置最大循环次数
    print(f"--- 循环 {i+1} ---\n")
    
    # 3.1. 构建Prompt
    full_prompt = "\n".join(prompt_history)
    
    # 3.2. 调用LLM进行思考
    response = llm.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": AGENT_SYSTEM_PROMPT},
            {"role": "user", "content": full_prompt}
        ]
    )
    llm_output = response.choices[0].message.content
    # 模型可能会输出多余的Thought-Action，需要截断
    match = re.search(r'(Thought:.*?Action:.*?)(?=\n\s*(?:Thought:|Action:|Observation:)|\Z)', llm_output, re.DOTALL)
    if match:
        truncated = match.group(1).strip()
        if truncated != llm_output.strip():
            llm_output = truncated
            print("已截断多余的 Thought-Action 对")
    print(f"模型输出:\n{llm_output}\n")
    prompt_history.append(llm_output)
    
    # 3.3. 解析并执行行动
    action_match = re.search(r"Action: (.*)", llm_output, re.DOTALL)
    if not action_match:
        print("解析错误:模型输出中未找到 Action。")
        break
    action_str = action_match.group(1).strip()

    if action_str.startswith("finish"):
        final_answer = re.search(r'finish\(answer="(.*)"\)', action_str).group(1)
        print(f"任务完成，最终答案: {final_answer}")
        break
    
    tool_name = re.search(r"(\w+)\(", action_str).group(1)
    args_str = re.search(r"\((.*)\)", action_str).group(1)
    kwargs = dict(re.findall(r'(\w+)="([^"]*)"', args_str))

    if tool_name in available_tools:
        observation = available_tools[tool_name](**kwargs)
    else:
        observation = f"错误:未定义的工具 '{tool_name}'"

    # 3.4. 记录观察结果
    observation_str = f"Observation: {observation}"
    print(f"{observation_str}\n" + "="*40)
    prompt_history.append(observation_str)

用户输入: 你好，请帮我查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点。
--- 循环 1 ---

模型输出:
Thought: 我需要先获取今天北京的实时天气，然后根据天气情况推荐合适的景点。下一步先调用查询北京天气的接口获取天气信息。
Action: get_weather(city="北京")

Observation: 错误:查询天气时遇到网络问题 - HTTPSConnectionPool(host='wttr.in', port=443): Max retries exceeded with url: /%E5%8C%97%E4%BA%AC?format=j1 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1032)')))
--- 循环 2 ---

模型输出:
Thought: 上次查询北京天气时遇到网络错误（SSL 连接中断）。我将重试一次获取今日北京的实时天气，以便随后根据天气推荐合适景点。如果再次失败，会告知用户并建议用户稍后重试或提供容错方案（如手动提供天气或允许我用常见情况进行推荐）。
Action: get_weather(city="北京")

Observation: 北京当前天气:Mist，气温-3摄氏度
--- 循环 3 ---

模型输出:
Thought: 我已经成功获取到北京当前天气为 Mist（薄雾），气温 -3°C。下一步根据“北京 + Mist”来查找适合在薄雾低温天气下游玩的景点（如室内或近距离观赏、能提供避风保暖的场所）。我将调用景点推荐接口获取具体建议。
Action: get_attraction(city="北京", weather="Mist")

Observation: In mist, visit the Forbidden City for history, the Temple of Heaven for architecture, and the Great Wall at Mutianyu for views.
--- 循环 4 ---

模型输出:
Thought: 我已经得到北京当前天气（

## GPT 优化版

主要优化的是结构化输出。有点受不了用re去匹配工具……

In [22]:
from __future__ import annotations

import os
import json
import traceback
from dataclasses import dataclass
from typing import Any, Callable, Dict, Optional, Tuple

from openai import OpenAI


# =========================
# 1) 初始化 OpenAI 客户端（按你给的风格，但修正语法）
# =========================
api_key = os.getenv("OPENAI_API_KEY")  # 你的 .env 或系统环境变量
base_url = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
model = os.getenv("OPENAI_MODEL", "gpt-5-nano")

print("OpenAI API Key:", api_key[:10] + "..." if api_key else "Not found")
print("OpenAI Base URL:", base_url)
print("OpenAI Model:", model)

if not api_key:
    raise RuntimeError("未检测到 OPENAI_API_KEY，请先设置环境变量")

# 关键点：model 不要传给 OpenAI(...) 构造函数
# model 应该在每次请求 create(...) 时传入
llm = OpenAI(
    api_key=api_key,
    base_url=base_url,
)

# =========================
# 2) Agent System Prompt（核心优化点：只允许输出 JSON）
# =========================
# 为什么这是关键优化点
# 你原来用正则，是因为模型输出自由文本
# 现在强制模型只输出一个 JSON 对象，解析直接 json.loads，稳定性提升很多
AGENT_SYSTEM_PROMPT = """
你是一个严格遵守协议的 Agent。
你必须且只能输出一个 JSON 对象，禁止输出任何额外文本，禁止使用 Markdown 代码块。

JSON 格式必须是：
{
  "thought": "简短思路概述，必须存在这个字段",
  "action": {
    "tool": "工具名，必须来自工具列表",
    "args": { "参数名": 参数值 }
  }
}

规则
1. 每次只输出一个 action，不允许一次输出多个 action
2. tool 必须来自工具列表
3. 当可以给出最终答复时，必须调用 finish 工具，args 必须包含 answer 字段
4. 如果需要外部信息，就调用 search_web 或 get_weather
5. 输出必须是可被 json.loads 解析的合法 JSON

工具列表
- get_weather: 查询天气
- search_web: 搜索信息
- recommend_spot: 根据天气推荐景点
- finish: 输出最终答案
""".strip()


# =========================
# 3) OpenAI 调用封装（核心优化点：把 SDK 调用藏在一个函数里）
# =========================
# 为什么要封装
# 1) Agent 只关心输入输出字符串，不关心 SDK 的细节
# 2) 以后你换 Responses API 或换模型，只改这一处
def llm_generate(llm_client: OpenAI, model: str, system_prompt: str, user_prompt: str) -> str:
    """
    返回模型的原始文本输出
    注意：Agent 协议要求输出必须是 JSON，这里不做清洗，交给解析层处理
    """
    resp = llm_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,  # 关键点：Agent 场景强烈建议 0，减少随机输出导致的协议破坏
    )
    return resp.choices[0].message.content or ""


# =========================
# 4) 工具层（代码世界的能力）
# =========================
# 关键理解点
# 模型不会真的去查天气，也不会真的去上网
# 模型只能选择工具名和参数
# 真正执行一定发生在这里

def tool_get_weather(city: str, date: str = "today") -> Dict[str, Any]:
    """
    示例天气工具
    真实项目你可以接入天气 API 或用 search_web 来查
    返回结构化数据，尽量不要返回散文
    """
    return {
        "city": city,
        "date": date,
        "condition": "晴",
        "temperature_c": 2,
        "wind": "西北风 3 级",
        "humidity_percent": 35,
    }


def tool_search_web(query: str) -> Dict[str, Any]:
    """
    示例搜索工具
    如果你要用 Tavily，就在这里接入
    现在先返回模拟结果，便于你跑通 Agent 主流程
    """
    tavily_key = os.getenv("TAVILY_API_KEY")
    if not tavily_key:
        return {
            "query": query,
            "warning": "未设置 TAVILY_API_KEY，当前为模拟结果",
            "results": [
                {"title": "模拟结果 1", "snippet": "这里是摘要", "url": "https://example.com/1"},
                {"title": "模拟结果 2", "snippet": "这里是摘要", "url": "https://example.com/2"},
            ],
        }

    # 这里留一个你未来接入的入口，避免你现在被 SDK 细节打断学习主线
    # 你接入后，保持返回结构化 JSON 即可
    return {
        "query": query,
        "note": "检测到 TAVILY_API_KEY，但此示例未实现真实联网搜索",
        "results": [],
    }


def tool_recommend_spot(city: str, weather: Dict[str, Any]) -> Dict[str, Any]:
    """
    示例推荐工具
    为什么要做这个工具
    你可以把部分可控逻辑从 LLM 中剥离出来，减少胡编风险
    也可以不用这个工具，让 LLM 直接 finish
    """
    condition = weather.get("condition", "")
    temp = weather.get("temperature_c", None)

    if "雨" in condition:
        spot = "国家博物馆或大型室内展馆"
        reason = "下雨天更适合室内，避免体验受影响"
    else:
        if temp is not None and temp <= 3:
            spot = "什刹海周边散步，搭配室内茶馆休息"
            reason = "天气晴但偏冷，短时户外加室内补给更舒适"
        else:
            spot = "颐和园或奥林匹克森林公园"
            reason = "晴朗适合户外大景点，步行与拍照体验好"

    return {
        "city": city,
        "spot": spot,
        "reason": reason,
        "based_on": {"condition": condition, "temperature_c": temp},
    }


def tool_finish(answer: str) -> Dict[str, Any]:
    """
    finish 工具只是一个协议终止信号
    执行器看到 tool == finish 就结束循环
    """
    return {"answer": answer}


# 工具白名单注册表（核心优化点：只允许调用这里的工具）
TOOLS: Dict[str, Callable[..., Any]] = {
    "get_weather": tool_get_weather,
    "search_web": tool_search_web,
    "recommend_spot": tool_recommend_spot,
    "finish": tool_finish,
}


# =========================
# 5) 解析与校验（核心优化点：把协议错误变成可回传的 Observation）
# =========================
def parse_agent_json(output: str) -> Tuple[Optional[Dict[str, Any]], Optional[str]]:
    """
    输出：data, error_message
    为什么要这么设计
    1) 不要解析失败就直接崩，这会让你调试很痛苦
    2) 把错误回传给模型，模型有机会修正 JSON 输出
    """
    try:
        data = json.loads(output)
    except json.JSONDecodeError as e:
        return None, f"JSONDecodeError: {e}"

    if not isinstance(data, dict):
        return None, "协议错误：顶层必须是 JSON object"

    if "thought" not in data or "action" not in data:
        return None, "协议错误：必须包含 thought 和 action 字段"

    action = data["action"]
    if not isinstance(action, dict):
        return None, "协议错误：action 必须是 JSON object"

    if "tool" not in action or "args" not in action:
        return None, "协议错误：action 必须包含 tool 和 args"

    tool = action["tool"]
    args = action["args"]

    if not isinstance(tool, str) or not tool:
        return None, "协议错误：tool 必须是非空字符串"

    if not isinstance(args, dict):
        return None, "协议错误：args 必须是 JSON object"

    return data, None


@dataclass
class AgentConfig:
    max_loops: int = 6
    debug_print: bool = True
    json_repair_retries: int = 2  # 关键优化点：给模型修 JSON 的次数


# =========================
# 6) Agent 执行器（JSON ReAct）
# =========================
class JSONReActAgent:
    def __init__(
        self,
        llm_client: OpenAI,
        model: str,
        tools: Dict[str, Callable[..., Any]],
        system_prompt: str,
        cfg: AgentConfig = AgentConfig(),
    ):
        self.llm = llm_client
        self.model = model
        self.tools = tools
        self.system_prompt = system_prompt
        self.cfg = cfg

    def _log(self, s: str) -> None:
        if self.cfg.debug_print:
            print(s)

    def run(self, user_prompt: str) -> str:
        """
        执行流程
        1) 把历史拼成 full_prompt
        2) 调 LLM 得到 JSON
        3) 解析 JSON 得到 tool 与 args
        4) 执行工具得到 observation
        5) observation 回灌给模型
        6) 直到 finish
        """
        history: list[Dict[str, str]] = []
        history.append({"role": "user", "content": user_prompt})

        for i in range(self.cfg.max_loops):
            self._log(f"\n====== LOOP {i + 1} ======")

            full_prompt = self._render_history(history)
            raw = llm_generate(
                llm_client=self.llm,
                model=self.model,
                system_prompt=self.system_prompt,
                user_prompt=full_prompt,
            )
            self._log(f"LLM output:\n{raw}")

            data, err = parse_agent_json(raw)

            # 关键优化点：解析失败时，不是立刻退出
            # 而是把错误当成 Observation 回传，让模型修正
            repair_count = 0
            while err is not None and repair_count < self.cfg.json_repair_retries:
                repair_count += 1

                obs = {
                    "error": "你的输出不是合法 JSON 或不符合协议，请只输出一个合法 JSON 对象",
                    "detail": err,
                    "example": {
                        "thought": "need weather",
                        "action": {"tool": "get_weather", "args": {"city": "北京", "date": "today"}},
                    },
                }

                history.append({"role": "assistant", "content": raw})
                history.append({"role": "observation", "content": json.dumps(obs, ensure_ascii=False)})

                full_prompt = self._render_history(history)
                raw = llm_generate(
                    llm_client=self.llm,
                    model=self.model,
                    system_prompt=self.system_prompt,
                    user_prompt=full_prompt,
                )
                self._log(f"LLM repaired output:\n{raw}")

                data, err = parse_agent_json(raw)

            if err is not None:
                return f"Agent 失败：模型多次输出无效 JSON。最后错误：{err}"

            tool = data["action"]["tool"]
            args = data["action"]["args"]
            thought = data.get("thought", "")

            self._log(f"Parsed thought: {thought}")
            self._log(f"Parsed action: tool={tool}, args={args}")

            # 关键优化点：工具白名单，模型乱写工具名也不会执行未知代码
            if tool not in self.tools:
                obs = {
                    "error": f"未定义的工具：{tool}",
                    "allowed_tools": list(self.tools.keys()),
                }
                history.append({"role": "assistant", "content": raw})
                history.append({"role": "observation", "content": json.dumps(obs, ensure_ascii=False)})
                continue

            # 执行工具
            try:
                result = self.tools[tool](**args)
            except TypeError as e:
                # 关键优化点：参数名不匹配时，把错误回传给模型修 args
                obs = {
                    "error": "工具调用参数错误",
                    "tool": tool,
                    "detail": str(e),
                    "hint": "请检查 args 的参数名是否与工具函数签名一致",
                }
                history.append({"role": "assistant", "content": raw})
                history.append({"role": "observation", "content": json.dumps(obs, ensure_ascii=False)})
                continue
            except Exception as e:
                obs = {
                    "error": "工具执行异常",
                    "tool": tool,
                    "detail": str(e),
                    "traceback": traceback.format_exc()[:2000],
                }
                history.append({"role": "assistant", "content": raw})
                history.append({"role": "observation", "content": json.dumps(obs, ensure_ascii=False)})
                continue

            self._log("Tool result:")
            self._log(json.dumps(result, ensure_ascii=False, indent=2))

            # 结束条件
            if tool == "finish":
                answer = result.get("answer", "")
                return answer if answer else "finish 未提供 answer"

            # 记录本轮 assistant 输出与 observation
            # 关键优化点：Observation 强制 JSON 字符串，减少模型误读
            history.append({"role": "assistant", "content": raw})
            history.append({"role": "observation", "content": json.dumps(result, ensure_ascii=False)})

        return "已达到最大循环次数，但模型未调用 finish。"

    def _render_history(self, history: list[Dict[str, str]]) -> str:
        """
        把历史渲染成一个字符串 prompt
        为什么要这样渲染
        你的 llm_generate 目前走的是 messages=[system,user] 的单轮接口
        所以我们需要把多轮 history 压成一个 user_prompt 文本
        """
        lines = []
        for item in history:
            role = item["role"]
            content = item["content"]
            if role == "user":
                lines.append(f"User: {content}")
            elif role == "assistant":
                lines.append(f"Assistant: {content}")
            else:
                lines.append(f"Observation: {content}")
        return "\n".join(lines)


# =========================
# 7) 运行示例
# =========================
if __name__ == "__main__":
    agent = JSONReActAgent(
        llm_client=llm,
        model=model,
        tools=TOOLS,
        system_prompt=AGENT_SYSTEM_PROMPT,
        cfg=AgentConfig(max_loops=6, debug_print=True, json_repair_retries=2),
    )

    user_prompt = "你好，请帮我查询一下今天苏州的天气，然后根据天气推荐一个合适的旅游景点。"
    print("\n用户输入:", user_prompt)

    final_answer = agent.run(user_prompt)
    print("\nFINAL ANSWER:")
    print(final_answer)



OpenAI API Key: sk-d7GDhmo...
OpenAI Base URL: https://xiaoai.plus/v1
OpenAI Model: gpt-5-nano

用户输入: 你好，请帮我查询一下今天苏州的天气，然后根据天气推荐一个合适的旅游景点。

====== LOOP 1 ======
LLM output:
{
  "thought": "先查询苏州今天的天气信息。",
  "action": {
    "tool": "get_weather",
    "args": { "city": "Suzhou", "date": "today" }
  }
}
Parsed thought: 先查询苏州今天的天气信息。
Parsed action: tool=get_weather, args={'city': 'Suzhou', 'date': 'today'}
Tool result:
{
  "city": "Suzhou",
  "date": "today",
  "condition": "晴",
  "temperature_c": 2,
  "wind": "西北风 3 级",
  "humidity_percent": 35
}

====== LOOP 2 ======
LLM output:
{
  "thought": "天气晴朗且气温较低，适合室内或半户外兼具观光与休息的景点。将基于当前天气信息推荐一个合适的景点。",
  "action": {
    "tool": "recommend_spot",
    "args": {
      "city": "Suzhou",
      "condition": "晴",
      "temperature_c": 2,
      "wind": "西北风 3 级",
      "humidity_percent": 35
    }
  }
}
Parsed thought: 天气晴朗且气温较低，适合室内或半户外兼具观光与休息的景点。将基于当前天气信息推荐一个合适的景点。
Parsed action: tool=recommend_spot, args={'city': 'Suzhou', 'condition': '晴', 'tempera